In [ ]:
!pip install requests bs4 lxml

Tạp chí KHCN

In [1]:
import requests
from bs4 import BeautifulSoup
import urllib3

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

def get_vjst_links(url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'
    }

    response = requests.get(url, headers=headers, verify=False)
    soup = BeautifulSoup(response.text, 'html.parser')

    found_links = []
    pdf_tags = soup.find_all('a', class_='obj_galley_link pdf')

    for tag in pdf_tags:
        view_url = tag['href']
        if '/view/' in view_url:
            download_url = view_url.replace('/view/', '/download/')
            found_links.append(download_url)

    return found_links

url_archive = "https://b.vjst.vn/index.php/ban_b"
links = get_vjst_links(url_archive)

print(f"Tim thay {len(links)} link PDF truc tiep.")

Tim thay 15 link PDF truc tiep.


In [2]:
import requests
from bs4 import BeautifulSoup
import urllib3
import time
import json

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

class VJST_JSON_Scraper:
    def __init__(self):
        self.headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'
        }
        self.source_name = "Tap chi Khoa hoc va Cong nghe Viet Nam (B)"
        self.journal_code = "vjst_b"
        self.issue_urls = set()
        self.articles_data = []

    def get_issue_list(self, max_pages=15):
        for page in range(1, max_pages + 1):
            archive_url = f"https://b.vjst.vn/index.php/ban_b/issue/archive?issuePage={page}"
            print(f"--- Dang lay danh sach so tu Archive trang {page} ---")
            try:
                res = requests.get(archive_url, headers=self.headers, verify=False, timeout=15)
                soup = BeautifulSoup(res.text, 'html.parser')
                links = soup.find_all('a', class_='title')

                if not links:
                    print("Het trang archive hoac khong tim thay them du lieu.")
                    break

                count_before = len(self.issue_urls)
                for a in links:
                    self.issue_urls.add(a['href'])

                if len(self.issue_urls) == count_before:
                    break
            except Exception as e:
                print(f"Loi tai trang {page}: {e}")
                break
        print(f"Tong cong tim thay {len(self.issue_urls)} so bao.")

    def crawl_articles(self):
        for idx, url in enumerate(self.issue_urls):
            print(f"[{idx+1}/{len(self.issue_urls)}] Dang boc tach bai viet tai: {url}")
            try:
                res = requests.get(url, headers=self.headers, verify=False, timeout=15)
                soup = BeautifulSoup(res.text, 'html.parser')
                article_summaries = soup.select('.obj_article_summary')

                for summary in article_summaries:
                    title_tag = summary.select_one('.title a')
                    pdf_tag = summary.select_one('a.obj_galley_link.pdf')

                    if title_tag and pdf_tag:
                        title = title_tag.get_text().strip()
                        pdf_url = pdf_tag['href'].replace('/view/', '/download/')

                        item = {
                            "source": self.source_name,
                            "journal_code": self.journal_code,
                            "article_title": title,
                            "pdf_url": pdf_url
                        }
                        self.articles_data.append(item)

                time.sleep(0.5)
            except Exception as e:
                print(f"Loi tai {url}: {e}")
                continue

    def save_json(self):
        output_file = f"{self.journal_code}_data.json"
        with open(output_file, "w", encoding="utf-8") as f:
            json.dump(self.articles_data, f, ensure_ascii=False, indent=4)
        print(f"Da luu {len(self.articles_data)} bai bao vao {output_file}")

bot = VJST_JSON_Scraper()
bot.get_issue_list()
bot.crawl_articles()
bot.save_json()

--- Dang lay danh sach so tu Archive trang 1 ---
--- Dang lay danh sach so tu Archive trang 2 ---
Tong cong tim thay 25 so bao.
[1/25] Dang boc tach bai viet tai: https://b.vjst.vn/index.php/ban_b/issue/view/126
[2/25] Dang boc tach bai viet tai: https://b.vjst.vn/index.php/ban_b/issue/view/132
[3/25] Dang boc tach bai viet tai: https://b.vjst.vn/index.php/ban_b/issue/view/130
[4/25] Dang boc tach bai viet tai: https://b.vjst.vn/index.php/ban_b/issue/view/135
[5/25] Dang boc tach bai viet tai: https://b.vjst.vn/index.php/ban_b/issue/view/136
[6/25] Dang boc tach bai viet tai: https://b.vjst.vn/index.php/ban_b/issue/view/121
[7/25] Dang boc tach bai viet tai: https://b.vjst.vn/index.php/ban_b/issue/view/122
[8/25] Dang boc tach bai viet tai: https://b.vjst.vn/index.php/ban_b/issue/view/137
[9/25] Dang boc tach bai viet tai: https://b.vjst.vn/index.php/ban_b/issue/view/131
[10/25] Dang boc tach bai viet tai: https://b.vjst.vn/index.php/ban_b/issue/view/133
[11/25] Dang boc tach bai viet 

Tạp chí giáo dục

In [3]:
import requests
from bs4 import BeautifulSoup
import urllib3
import time
import json

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

class TCGiaoDuc_JSON_Scraper:
    def __init__(self):
        self.headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'
        }
        self.source_name = "Tap chi Giao duc"
        self.journal_code = "tcgd"
        self.issue_urls = set()
        self.articles_data = []

    def get_issue_list(self, max_pages=20):
        for page in range(1, max_pages + 1):
            archive_url = f"https://tcgd.tapchigiaoduc.edu.vn/index.php/tapchi/issue/archive/{page}"
            print(f"--- Dang quet Archive trang {page} ---")
            try:
                res = requests.get(archive_url, headers=self.headers, verify=False, timeout=20)
                soup = BeautifulSoup(res.text, 'html.parser')
                links = soup.find_all('a', class_='title')

                if not links:
                    print("Het trang archive hoac khong thay so bao moi.")
                    break

                count_before = len(self.issue_urls)
                for a in links:
                    self.issue_urls.add(a['href'])

                if len(self.issue_urls) == count_before:
                    break

                time.sleep(0.3)
            except Exception as e:
                print(f"Loi archive trang {page}: {e}")
                break
        print(f"Da tim thay {len(self.issue_urls)} so tap chi.")

    def crawl_articles(self):
        for idx, url in enumerate(self.issue_urls):
            print(f"[{idx+1}/{len(self.issue_urls)}] Boc tach: {url}")
            try:
                res = requests.get(url, headers=self.headers, verify=False, timeout=20)
                soup = BeautifulSoup(res.text, 'html.parser')
                summaries = soup.select('.obj_article_summary')

                for summary in summaries:
                    title_tag = summary.select_one('.title a')
                    pdf_tag = summary.select_one('a.obj_galley_link.pdf')

                    if title_tag and pdf_tag:
                        title = title_tag.get_text().strip()
                        raw_pdf_url = pdf_tag['href']
                        pdf_url = raw_pdf_url.replace('/view/', '/download/') if '/view/' in raw_pdf_url else raw_pdf_url

                        self.articles_data.append({
                            "source": self.source_name,
                            "journal_code": self.journal_code,
                            "article_title": title,
                            "pdf_url": pdf_url
                        })

                time.sleep(0.3)
            except Exception as e:
                print(f"Loi tai {url}: {e}")
                continue

    def save_to_json(self):
        output_file = f"{self.journal_code}_data.json"
        with open(output_file, "w", encoding="utf-8") as f:
            json.dump(self.articles_data, f, ensure_ascii=False, indent=4)
        print(f"Da luu {len(self.articles_data)} bai bao vao {output_file}")

if __name__ == "__main__":
    bot = TCGiaoDuc_JSON_Scraper()
    bot.get_issue_list(max_pages=30)
    bot.crawl_articles()
    bot.save_to_json()

--- Dang quet Archive trang 1 ---
--- Dang quet Archive trang 2 ---
--- Dang quet Archive trang 3 ---
--- Dang quet Archive trang 4 ---
--- Dang quet Archive trang 5 ---
--- Dang quet Archive trang 6 ---
--- Dang quet Archive trang 7 ---
--- Dang quet Archive trang 8 ---
Het trang archive hoac khong thay so bao moi.
Da tim thay 152 so tap chi.
[1/152] Boc tach: https://tcgd.tapchigiaoduc.edu.vn/index.php/tapchi/issue/view/68
[2/152] Boc tach: https://tcgd.tapchigiaoduc.edu.vn/index.php/tapchi/issue/view/81
[3/152] Boc tach: https://tcgd.tapchigiaoduc.edu.vn/index.php/tapchi/issue/view/140
[4/152] Boc tach: https://tcgd.tapchigiaoduc.edu.vn/index.php/tapchi/issue/view/116
[5/152] Boc tach: https://tcgd.tapchigiaoduc.edu.vn/index.php/tapchi/issue/view/53
[6/152] Boc tach: https://tcgd.tapchigiaoduc.edu.vn/index.php/tapchi/issue/view/66
[7/152] Boc tach: https://tcgd.tapchigiaoduc.edu.vn/index.php/tapchi/issue/view/155
[8/152] Boc tach: https://tcgd.tapchigiaoduc.edu.vn/index.php/tapchi/i

KH trẻ

In [4]:
import requests
from bs4 import BeautifulSoup
import re
import json
import time

class KhoaHocTre_JSON_Scraper:
    def __init__(self):
        self.headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36'
        }
        self.source_name = "Tap chi Khoa hoc Tre"
        self.journal_code = "khoahoctre"
        self.target_url = "http://tapchikhoahoctre.com/a-homepage-section/"
        self.articles_data = []

    def convert_drive_link(self, url):
        file_id_match = re.search(r'/file/d/([a-zA-Z0-9_-]+)', url)
        if file_id_match:
            file_id = file_id_match.group(1)
            return f"https://drive.google.com/uc?export=download&id={file_id}"

        if "/drive/folders/" in url:
            return url

        return None

    def crawl(self):
        print(f"--- Dang ket noi toi: {self.target_url} ---")
        try:
            response = requests.get(self.target_url, headers=self.headers, timeout=25)
            soup = BeautifulSoup(response.text, 'html.parser')

            links = soup.find_all('a', href=True)

            for link in links:
                href = link['href']
                text = link.get_text().strip()

                if "drive.google.com" in href:
                    direct_url = self.convert_drive_link(href)

                    if direct_url:
                        clean_title = " ".join(text.split()) if text else "Untitled Article"

                        self.articles_data.append({
                            "source": self.source_name,
                            "journal_code": self.journal_code,
                            "article_title": clean_title,
                            "pdf_url": direct_url
                        })
                        print(f"    + Da lay: {clean_title}")

        except Exception as e:
            print(f"Loi: {e}")

    def save_json(self):
        output_file = f"{self.journal_code}_data.json"
        with open(output_file, "w", encoding="utf-8") as f:
            json.dump(self.articles_data, f, ensure_ascii=False, indent=4)
        print(f"XONG! Da luu {len(self.articles_data)} muc vao {output_file}")

if __name__ == "__main__":
    bot = KhoaHocTre_JSON_Scraper()
    bot.crawl()
    bot.save_json()

--- Dang ket noi toi: http://tapchikhoahoctre.com/a-homepage-section/ ---
    + Da lay: SỐ XUẤT BẢN 2024 (4)
    + Da lay: SỐ XUẤT BẢN 2024 (3)
    + Da lay: SỐ XUẤT BẢN 2024 (2)
    + Da lay: SỐ XUẤT BẢN 2024 (1)
    + Da lay: SỐ XUẤT BẢN 2023 (4)
    + Da lay: SỐ XUẤT BẢN 2023 (3)
    + Da lay: SỐ XUẤT BẢN 2023 (2)
    + Da lay: SỐ XUẤT BẢN 2023 (1)
    + Da lay: SỐ XUẤT BẢN 2022 (4)
    + Da lay: SỐ XUẤT BẢN 2022 (3)
    + Da lay: SỐ XUẤT BẢN 2022 (2)
    + Da lay: SỐ XUẤT BẢN 2022 (1)
    + Da lay: SỐ XUẤT BẢN 2021 (4)
    + Da lay: Untitled Article
    + Da lay: SỐ XUẤT BẢN 2021 (3)
    + Da lay: SỐ XUẤT BẢN 2021 (2)
    + Da lay: SỐ XUẤT BẢN 2021 (1)
    + Da lay: SỐ XUẤT BẢN 2020 (3)
    + Da lay: SỐ XUẤT BẢN 2020 (1)
    + Da lay: SỐ XUẤT BẢN 2019 (2)
    + Da lay: SỐ XUẤT BẢN 2019 (1)
    + Da lay: Untitled Article
    + Da lay: SỐ XUẤT BẢN 2018 (3)
    + Da lay: SỐ XUẤT BẢN 2018 (2)
    + Da lay: SỐ XUẤT BẢN 2018 (1)
    + Da lay: SỐ XUẤT BẢN 2017 (3)
    + Da lay: SỐ XUẤT BẢ

In [5]:
import json

def filter_scientific_articles(input_file, output_file):
    blacklist = [
        "THE LE",
        "QUY DINH",
        "MUC LUC",
        "BIA TAP CHI",
        "HOI DONG",
        "PHU LUC",
        "THU TOA SOAN",
        "CHI SO",
        "THONG TIN CHUNG"
    ]

    try:
        with open(input_file, 'r', encoding='utf-8') as f:
            data = json.load(f)

        original_count = len(data)

        filtered_data = [
            item for item in data
            if not any(word in item["article_title"].upper() for word in blacklist)
        ]

        with open(output_file, 'w', encoding='utf-8') as f:
            json.dump(filtered_data, f, ensure_ascii=False, indent=4)

        removed_count = original_count - len(filtered_data)
        print(f"Da loc xong!")
        print(f"Tong so muc goc: {original_count}")
        print(f"Da xoa: {removed_count} muc (The le, Quy dinh...)")
        print(f"So bai bao thuc thu con lai: {len(filtered_data)}")

    except Exception as e:
        print(f"Loi: {e}")

filter_scientific_articles("khoahoctre_data.json", "khoahoctre_final.json")

Da loc xong!
Tong so muc goc: 34
Da xoa: 0 muc (The le, Quy dinh...)
So bai bao thuc thu con lai: 34


Tạp chí Khoa học và Công nghệ Nhiệt đới

In [6]:
import requests
import re
from bs4 import BeautifulSoup
import urllib3
import time
import json

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

class VRTC_Standard_JSON_Scraper:
    def __init__(self):
        self.session = requests.Session()
        self.headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36',
        }
        self.source_name = "Tap chi Y sinh hoc nhiet doi (VRTC)"
        self.journal_code = "vrtc"
        self.base_url = "http://vrtc.org.vn"
        self.results = []

    def bypass_security(self, url):
        try:
            res = self.session.get(url, headers=self.headers, verify=False, timeout=15)
            cookie_match = re.search(r'document\.cookie="D1N=([a-f0-9]+)"', res.text)
            if cookie_match:
                token = cookie_match.group(1)
                self.session.cookies.set("D1N", token, domain="vrtc.org.vn")
                return True
        except: pass
        return False

    def get_article_data(self, art_url):
        try:
            res = self.session.get(art_url, headers=self.headers, verify=False, timeout=15)
            soup = BeautifulSoup(res.text, 'html.parser')

            title_tag = soup.find('h1', class_='post-title')
            article_title = title_tag.get_text().strip() if title_tag else "Untitled Article"

            pdf_match = re.search(r'https?://vrtc\.org\.vn/wp-content/uploads/[\d/]+/[^"\']+\.pdf', res.text)

            if pdf_match:
                pdf_url = pdf_match.group(0)
                return {
                    "source": self.source_name,
                    "journal_code": self.journal_code,
                    "article_title": article_title,
                    "pdf_url": pdf_url
                }
        except:
            pass
        return None

    def run(self):
        main_cat = f"{self.base_url}/y-sinh-nhiet-doi"
        print(f"--- Dang khoi tao va vuot bao mat VRTC ---")

        if not self.bypass_security(main_cat):
            print("Khong vuot duoc rao can D1N.")
            return

        res_main = self.session.get(main_cat, headers=self.headers, verify=False)
        soup_main = BeautifulSoup(res_main.text, 'html.parser')
        issue_links = [a['href'] for a in soup_main.find_all('a', href=True) if '/y-sinh-hoc-so-' in a['href']]
        issue_links = sorted(list(set(issue_links)))

        print(f"Tim thay {len(issue_links)} so bao. Dang boc tach bai viet...")

        for issue_url in issue_links:
            print(f"--- Dang quet: {issue_url.split('/')[-1].upper()} ---")
            try:
                res_issue = self.session.get(issue_url, headers=self.headers, verify=False)
                soup_issue = BeautifulSoup(res_issue.text, 'html.parser')
                article_links = [a['href'] for a in soup_issue.select('h2.post-box-title a')]

                for art_url in article_links:
                    data = self.get_article_data(art_url)
                    if data:
                        if not any(item['pdf_url'] == data['pdf_url'] for item in self.results):
                            self.results.append(data)
                            print(f"    + {data['article_title'][:60]}...")

                    time.sleep(0.2)
            except:
                continue

        output_file = f"{self.journal_code}_data.json"
        with open(output_file, "w", encoding="utf-8") as f:
            json.dump(self.results, f, ensure_ascii=False, indent=4)

        print(f"XONG! Da luu {len(self.results)} bai bao vao {output_file}")

if __name__ == "__main__":
    scraper = VRTC_Standard_JSON_Scraper()
    scraper.run()

--- Dang khoi tao va vuot bao mat VRTC ---
Tim thay 29 so bao. Dang boc tach bai viet...
--- Dang quet: Y-SINH-HOC-SO-01 ---
    + Kết quả ứng dụng chế phẩm Peptit điều hòa sinh học hỗ trợ đi...
    + Khảo sát nồng độ Dioxin trong nhau thai và sữa mẹ ở một số đ...
--- Dang quet: Y-SINH-HOC-SO-02 ---
    + Nhập môn độc học nhiệt đới...
    + Về nhiệm vụ nghiên cứu của Hướng Y sinh nhiệt đới tại Trung ...
    + Kết quả ứng dụng chế phẩm peptit điều hoà sinh học hỗ trợ đi...
    + Nghiên cứu đa hình Gen CYP1A1, GSTM1, GSTT1 và tình trạng su...
--- Dang quet: Y-SINH-HOC-SO-03 ---
    + Bảo vệ sức khoẻ con người trong điều kiện khắc nghiệt...
    + Молекулярные методы исследования стрептококков группы В...
    + Điều tra sơ bộ các bệnh tim mạch của quân nhân và công nhân ...
    + Một số quan điểm trong điều trị bệnh cúm A/H1N1 ở Liên bang ...
--- Dang quet: Y-SINH-HOC-SO-04 ---
    + Sự rối loạn nhịp sinh học và độc học miễn dịch benzapiren...
    + Hiệu quả điều trị điếc đột ngột bằng kết

In [7]:
import requests
import re
from bs4 import BeautifulSoup
import urllib3
import time
import json

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

class VRTCSinhThai_JSON_Scraper:
    def __init__(self):
        self.session = requests.Session()
        self.headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36',
        }
        self.source_name = "Tap chi Sinh thai nhiet doi (VRTC)"
        self.journal_code = "vrtc_sinh_thai"
        self.base_url = "http://vrtc.org.vn"
        self.results = []

    def bypass_security(self, url):
        try:
            res = self.session.get(url, headers=self.headers, verify=False, timeout=15)
            cookie_match = re.search(r'document\.cookie="D1N=([a-f0-9]+)"', res.text)
            if cookie_match:
                token = cookie_match.group(1)
                self.session.cookies.set("D1N", token, domain="vrtc.org.vn")
                return True
        except: pass
        return False

    def get_article_data(self, art_url):
        try:
            res = self.session.get(art_url, headers=self.headers, verify=False, timeout=15)
            soup = BeautifulSoup(res.text, 'html.parser')

            title_tag = soup.find('h1', class_='post-title')
            article_title = title_tag.get_text().strip() if title_tag else "Untitled Article"

            pdf_match = re.search(r'https?://vrtc\.org\.vn/wp-content/uploads/[\d/]+/[^"\']+\.pdf', res.text)

            if pdf_match:
                return {
                    "source": self.source_name,
                    "journal_code": self.journal_code,
                    "article_title": article_title,
                    "pdf_url": pdf_match.group(0)
                }
        except: pass
        return None

    def run(self):
        target_url = f"{self.base_url}/sinh-thai-nhiet-doi"
        print(f"--- Dang khoi tao bao mat cho chuyen muc Sinh Thai ---")

        if not self.bypass_security(target_url):
            print("Khong vuot duoc rao can D1N.")
            return

        res_main = self.session.get(target_url, headers=self.headers, verify=False)
        soup_main = BeautifulSoup(res_main.text, 'html.parser')

        issue_links = [a['href'] for a in soup_main.find_all('a', href=True)
                      if '/chuyen/sinh-thai' in a['href'] and '-so-' in a['href']]
        issue_links = sorted(list(set(issue_links)))

        print(f"Tim thay {len(issue_links)} so bao Sinh Thai. Dang boc tach...")

        for issue_url in issue_links:
            issue_name = issue_url.split('/')[-1].upper()
            print(f"--- Dang quet so: {issue_name} ---")
            try:
                res_issue = self.session.get(issue_url, headers=self.headers, verify=False)
                soup_issue = BeautifulSoup(res_issue.text, 'html.parser')
                article_links = [a['href'] for a in soup_issue.select('h2.post-box-title a')]

                for art_url in article_links:
                    data = self.get_article_data(art_url)
                    if data:
                        if not any(item['pdf_url'] == data['pdf_url'] for item in self.results):
                            self.results.append(data)
                            print(f"    + {data['article_title'][:60]}...")

                    time.sleep(0.2)
            except: continue

        output_file = f"{self.journal_code}_data.json"
        with open(output_file, "w", encoding="utf-8") as f:
            json.dump(self.results, f, ensure_ascii=False, indent=4)

        print(f"HOAN THANH! Da luu {len(self.results)} bai bao Sinh Thai vao {output_file}")

if __name__ == "__main__":
    scraper = VRTCSinhThai_JSON_Scraper()
    scraper.run()

--- Dang khoi tao bao mat cho chuyen muc Sinh Thai ---
Tim thay 30 so bao Sinh Thai. Dang boc tach...
--- Dang quet so: ST-CNSH-SO-01 ---
    + Kết quả chủ yếu và định hướng hoạt động nghiên cứu khoa học ...
    + Sự biến đổi đa dạng sinh học do tác động của con người và an...
    + Định hình hướng sinh thái cạn tại Trung tâm Nhiệt đới Việt –...
    + Đặc điểm thực vật rừng nhiệt đới gió mùa Việt Nam...
    + Một số đặc điểm sinh học sinh sản của cá Phèn sọc đen (Upene...
    + Nghiên cứu một số đặc điểm của khu hệ cá rạn san hô tại khu ...
    + Một số kết quả nghiên cứu hiện tượng nội ký sinh giống Ichth...
    + Cơ sở lý thuyết của phương pháp phương sai rối trong nghiên ...
--- Dang quet so: ST-CNSH-SO-02 ---
--- Dang quet so: ST-CNSH-SO-03 ---
    + Ảnh hưởng của selen hữu cơ lên sinh trưởng, tỷ lệ sống, thàn...
    + Hàm lượng kim loại nặng trong một số loài cá kinh tế vùng bi...
--- Dang quet so: ST-CNSH-SO-04 ---
    + Đánh giá tác động của vi sinh vật đến chất lượng nhiên liệu

In [8]:
import requests
import re
from bs4 import BeautifulSoup
import urllib3
import time
import json

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

class VRTCDoBen_JSON_Scraper:
    def __init__(self):
        self.session = requests.Session()
        self.headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36',
        }
        self.source_name = "Tap chi Do ben nhiet doi (VRTC)"
        self.journal_code = "vrtc_do_ben"
        self.base_url = "http://vrtc.org.vn"
        self.results = []

    def bypass_security(self, url):
        try:
            res = self.session.get(url, headers=self.headers, verify=False, timeout=15)
            cookie_match = re.search(r'document\.cookie="D1N=([a-f0-9]+)"', res.text)
            if cookie_match:
                token = cookie_match.group(1)
                self.session.cookies.set("D1N", token, domain="vrtc.org.vn")
                return True
        except: pass
        return False

    def get_article_data(self, art_url):
        try:
            res = self.session.get(art_url, headers=self.headers, verify=False, timeout=15)
            soup = BeautifulSoup(res.text, 'html.parser')

            title_tag = soup.find('h1', class_='post-title')
            article_title = title_tag.get_text().strip() if title_tag else "Untitled Article"

            pdf_match = re.search(r'https?://vrtc\.org\.vn/wp-content/uploads/[\d/]+/[^"\']+\.pdf', res.text)

            if pdf_match:
                return {
                    "source": self.source_name,
                    "journal_code": self.journal_code,
                    "article_title": article_title,
                    "pdf_url": pdf_match.group(0)
                }
        except: pass
        return None

    def run(self):
        target_url = f"{self.base_url}/do-ben-nhiet-doi"
        print(f"--- Dang khoi tao bao mat cho chuyen muc Do Ben ---")

        if not self.bypass_security(target_url):
            print("Khong vuot duoc rao can bao mat D1N.")
            return

        res_main = self.session.get(target_url, headers=self.headers, verify=False)
        soup_main = BeautifulSoup(res_main.text, 'html.parser')

        issue_links = [a['href'] for a in soup_main.find_all('a', href=True)
                      if ('/chuyen/do-ben' in a['href'] or '/chuyen/hoa-hoc' in a['href']) and '-so-' in a['href']]
        issue_links = sorted(list(set(issue_links)))

        print(f"Tim thay {len(issue_links)} so bao Do Ben. Dang xu ly...")

        for issue_url in issue_links:
            issue_name = issue_url.split('/')[-1].upper()
            print(f"--- Dang quet so: {issue_name} ---")
            try:
                res_issue = self.session.get(issue_url, headers=self.headers, verify=False)
                soup_issue = BeautifulSoup(res_issue.text, 'html.parser')
                article_links = [a['href'] for a in soup_issue.select('h2.post-box-title a')]

                for art_url in article_links:
                    data = self.get_article_data(art_url)
                    if data:
                        if not any(item['pdf_url'] == data['pdf_url'] for item in self.results):
                            self.results.append(data)
                            print(f"    + {data['article_title'][:60]}...")

                    time.sleep(0.2)
            except: continue

        output_file = f"{self.journal_code}_data.json"
        with open(output_file, "w", encoding="utf-8") as f:
            json.dump(self.results, f, ensure_ascii=False, indent=4)

        print(f"HOAN THANH! Da thu hoach {len(self.results)} bai bao Do Ben vao {output_file}")

if __name__ == "__main__":
    scraper = VRTCDoBen_JSON_Scraper()
    scraper.run()

--- Dang khoi tao bao mat cho chuyen muc Do Ben ---
Tim thay 29 so bao Do Ben. Dang xu ly...
--- Dang quet so: HH-VL-SO-01 ---
    + Защита от морской коррозии сталей в замкнутых объемах...
    + Khảo sát một số tính chất điện hóa của anốt Magnetit được ch...
    + Thử nghiệm lớp phủ bảo vệ cao su có chứa Nano Cacbon...
    + Nghiên cứu quy trình chế tạo chất cháy quân sự TPA...
    + Các chất ức chế ăn mòn kim loại ИФХАН...
    + Kết quả ban đầu trong ứng dụng chất chuyển hóa thứ cấp của c...
    + R-40 – Công nghệ xử lý bom đạn không gây nổ...
--- Dang quet so: HH-VL-SO-02 ---
    + Ý nghĩa về mặt phương pháp luận của các trạm thử nghiệm khí ...
    + Đánh giá khả năng ức chế ăn mòn thép và nhôm của một số dãy ...
    + Dự báo thời hạn bảo quản thuốc phóng pyrocxilin trong điều k...
    + Sự ức chế ăn mòn thép CT3 trong môi trường axít HCl 2M của 2...
    + Một số kết quả nghiên cứu mới của Hướng Độ bền nhiệt đới tại...
    + Thiết bị kiểm tra toàn bộ máy bay và kiểm tra trạng thái c

Tạp chí KHXH VN

In [9]:
import requests
from bs4 import BeautifulSoup
import urllib3
import time
import json

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

class KHXH_VN_JSON_Scraper:
    def __init__(self):
        self.headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36'
        }
        self.source_name = "Tap chi Khoa hoc xa hoi Viet Nam"
        self.journal_code = "khxhvn"
        self.results = []

    def get_direct_pdf(self, landing_url):
        try:
            res = requests.get(landing_url, headers=self.headers, verify=False, timeout=15)
            soup = BeautifulSoup(res.text, 'html.parser')

            download_link = soup.find('a', class_='download')
            if download_link:
                return download_link['href']

            iframe = soup.find('iframe', src=True)
            if iframe and '.pdf' in iframe['src']:
                return iframe['src']
        except:
            pass
        return None

    def crawl_issue(self, issue_url):
        print(f"--- Dang quet so: {issue_url.split('/')[-1]} ---")
        try:
            res = requests.get(issue_url, headers=self.headers, verify=False, timeout=20)
            soup = BeautifulSoup(res.text, 'html.parser')
            articles = soup.select('.obj_article_summary')

            for art in articles:
                title_tag = art.select_one('.title a')
                pdf_page_tag = art.select_one('a.obj_galley_link.pdf')

                if title_tag and pdf_page_tag:
                    article_title = title_tag.get_text().strip()
                    landing_url = pdf_page_tag['href']

                    if '/view/' in landing_url:
                        direct_pdf = landing_url.replace('/view/', '/download/')
                    else:
                        direct_pdf = self.get_direct_pdf(landing_url)

                    if direct_pdf:
                        self.results.append({
                            "source": self.source_name,
                            "journal_code": self.journal_code,
                            "article_title": article_title,
                            "pdf_url": direct_pdf
                        })
                        print(f"    + {article_title[:60]}...")

            time.sleep(0.5)
        except Exception as e:
            print(f"Loi khi quet so bao: {e}")

    def run(self, archive_url, max_pages=5):
        print(f"--- Dang lay danh sach so bao tu: {archive_url} ---")
        try:
            res = requests.get(archive_url, headers=self.headers, verify=False)
            soup = BeautifulSoup(res.text, 'html.parser')

            issue_links = [a['href'] for a in soup.find_all('a', class_='title') if '/issue/view/' in a['href']]
            issue_links = sorted(list(set(issue_links)), reverse=True)

            print(f"Tim thay {len(issue_links)} so bao cu.")

            for link in issue_links:
                self.crawl_issue(link)

            output_file = f"{self.journal_code}_data.json"
            with open(output_file, "w", encoding="utf-8") as f:
                json.dump(self.results, f, ensure_ascii=False, indent=4)

            print(f"HOAN THANH! Da luu {len(self.results)} bai bao vao {output_file}")

        except Exception as e:
            print(f"Loi he thong: {e}")

if __name__ == "__main__":
    scraper = KHXH_VN_JSON_Scraper()
    archive_base_url = "https://vjol.info.vn/index.php/khxhvn/issue/archive"
    scraper.run(archive_base_url)

--- Dang lay danh sach so bao tu: https://vjol.info.vn/index.php/khxhvn/issue/archive ---
Tim thay 25 so bao cu.
--- Dang quet so: 9837 ---
    + Từ bài viết Học tập suốt đời của Tổng Bí thư Tô Lâm nghĩ về ...
    + Đại hội XIV: bước chuyển biến mạnh mẽ trong tư duy chính trị...
    + Trí tuệ nhân tạo: một thách thức đối với triết học nhận thức...
    + Bàn luận về nguồn vốn xã hội và vốn xã hội...
    + Sử dụng khoa học kỹ thuật và công nghệ số trong đời sống nhì...
    + Tri thức hóa nông dân Việt Nam hiện nay: thách thức và giải ...
    + Chính sách phát triển giáo viên trung học phổ thông tại tỉnh...
    + Tích hợp ESG trong quản lý dự trữ ngoại hối của Ngân hàng Tr...
    + Chính sách nguồn nhân lực kỹ năng số: mô hình quốc tế và bài...
    + Pháp luật về kiểm soát tiếng ồn: kinh nghiệm của Nhật Bản và...
    + Vài nét về chế độ tập ấm đối với công thần của triều Nguyễn ...
    + Người A Rem ở Quảng Bình: từ cuộc sống hang đá đến định cư l...
    + Thực trạng phụ huynh hóa ở trẻ v

Tạp chí Khoa học xã hội miền Trung

In [10]:
import requests
import re
from bs4 import BeautifulSoup
import urllib3
import time
import json

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

class KHXHMienTrung_JSON_Scraper:
    def __init__(self):
        self.headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36'
        }
        self.source_name = "Tap chi Khoa hoc xa hoi mien Trung"
        self.journal_code = "khxh_mientrung"
        self.base_domain = "https://vjol.info.vn"
        self.results = []

    def get_direct_pdf_url(self, view_url):
        try:
            if '/view/' in view_url:
                return view_url.replace('/view/', '/download/')

            res = requests.get(view_url, headers=self.headers, verify=False, timeout=15)
            soup = BeautifulSoup(res.text, 'html.parser')

            download_btn = soup.find('a', class_='download')
            if download_btn:
                return download_btn['href']

            iframe = soup.find('iframe', src=True)
            if iframe and '.pdf' in iframe['src'].lower():
                return iframe['src']
        except:
            pass
        return None

    def run(self):
        archive_url = f"{self.base_domain}/index.php/isscr/issue/archive"
        print(f"--- Dang lay danh sach cac so bao tu: {archive_url} ---")

        try:
            res_archive = requests.get(archive_url, headers=self.headers, verify=False)
            soup_archive = BeautifulSoup(res_archive.text, 'html.parser')

            issue_links = [a['href'] for a in soup_archive.find_all('a', class_='title') if '/issue/view/' in a['href']]
            print(f"Tim thay {len(issue_links)} so bao.")

            for issue_url in issue_links:
                print(f"--- Dang quet so bao: {issue_url.split('/')[-1]} ---")
                res_issue = requests.get(issue_url, headers=self.headers, verify=False)
                soup_issue = BeautifulSoup(res_issue.text, 'html.parser')

                articles = soup_issue.select('.obj_article_summary')

                for art in articles:
                    title_tag = art.select_one('.title a')
                    pdf_tag = art.select_one('a.obj_galley_link.pdf')

                    if title_tag and pdf_tag:
                        article_title = title_tag.get_text().strip()
                        view_url = pdf_tag['href']
                        direct_link = self.get_direct_pdf_url(view_url)

                        if direct_link:
                            self.results.append({
                                "source": self.source_name,
                                "journal_code": self.journal_code,
                                "article_title": article_title,
                                "pdf_url": direct_link
                            })
                            print(f"    + {article_title[:60]}...")

                time.sleep(0.5)

            output_file = f"{self.journal_code}_data.json"
            with open(output_file, "w", encoding="utf-8") as f:
                json.dump(self.results, f, ensure_ascii=False, indent=4)

            print(f"Xong! Tong cong thu duoc {len(self.results)} bai bao vao {output_file}")

        except Exception as e:
            print(f"Loi he thong: {e}")

if __name__ == "__main__":
    bot = KHXHMienTrung_JSON_Scraper()
    bot.run()

--- Dang lay danh sach cac so bao tu: https://vjol.info.vn/index.php/isscr/issue/archive ---
Tim thay 25 so bao.
--- Dang quet so bao: 10031 ---
    + Ngoại giao khoa học – Thực tiễn của Liên bang Nga và khuyến ...
    + Tiếp nhận và tiếp biến tri thức khoa học, kỹ thuật phương Tâ...
    + Tác động bất đối xứng và theo thời gian của các yếu tố kinh ...
    + Liên kết giữa cơ sở giáo dục đại học và doanh nghiệp trong đ...
    + Tri thức địa phương trong phát triển sinh kế của các dân tộc...
    + Giáo dục và thực hành văn hóa truyền thống cho học sinh tron...
    + Tình hình thực hiện chính sách phát triển cán bộ Đoàn T...
    + Một số giải pháp ngăn ngừa xung đột xã hội trên địa bàn  tỉn...
    + Phát triển nông nghiệp công nghệ cao - Nhân tố quan trọng tr...
    + Phát triển hợp tác xã vùng Kinh tế trọng điểm miền Trung tro...
    + Ảnh hưởng của nhận thức về chất lượng dịch vụ khám chữa bệnh...
    + Chuyển đổi số và kinh doanh bền vững  tại các ngân hàng thươ...
    + Liên kết,

Tạp chí nghiên cứu lịch sử

In [11]:
import requests
from bs4 import BeautifulSoup
import urllib3
import time
import json

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

class HistoryResearch_JSON_Scraper:
    def __init__(self):
        self.headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36'
        }
        self.base_url = "https://vjol.info.vn"
        self.source_name = "Tap chi Nghien cuu Lich su"
        self.journal_code = "hists"
        self.results = []

    def get_direct_pdf(self, landing_url):
        try:
            if '/view/' in landing_url:
                return landing_url.replace('/view/', '/download/')

            res = requests.get(landing_url, headers=self.headers, verify=False, timeout=15)
            soup = BeautifulSoup(res.text, 'html.parser')

            download_link = soup.find('a', class_='download')
            if download_link:
                return download_link['href']

            iframe = soup.find('iframe', src=True)
            if iframe and '.pdf' in iframe['src'].lower():
                return iframe['src']
        except:
            pass
        return None

    def run(self):
        archive_url = f"{self.base_url}/index.php/{self.journal_code}/issue/archive"
        print(f"--- Dang quet danh muc so cu: {archive_url} ---")

        try:
            res_archive = requests.get(archive_url, headers=self.headers, verify=False)
            soup_archive = BeautifulSoup(res_archive.text, 'html.parser')

            issue_links = [a['href'] for a in soup_archive.find_all('a', class_='title') if '/issue/view/' in a['href']]
            print(f"Tim thay {len(issue_links)} so bao.")

            for issue_url in issue_links:
                print(f"--- Dang boc tach so: {issue_url.split('/')[-1]} ---")
                res_issue = requests.get(issue_url, headers=self.headers, verify=False)
                soup_issue = BeautifulSoup(res_issue.text, 'html.parser')

                articles = soup_issue.select('.obj_article_summary')

                for art in articles:
                    title_tag = art.select_one('.title a')
                    pdf_tag = art.select_one('a.obj_galley_link.pdf')

                    if title_tag and pdf_tag:
                        article_title = title_tag.get_text().strip()
                        landing_url = pdf_tag['href']
                        direct_pdf = self.get_direct_pdf(landing_url)

                        if direct_pdf:
                            self.results.append({
                                "source": self.source_name,
                                "journal_code": self.journal_code,
                                "article_title": article_title,
                                "pdf_url": direct_pdf
                            })
                            print(f"    + {article_title[:60]}...")

                time.sleep(0.4)

            output_name = f"{self.journal_code}_data.json"
            with open(output_name, "w", encoding="utf-8") as f:
                json.dump(self.results, f, ensure_ascii=False, indent=4)

            print(f"HOAN THANH! Thu duoc {len(self.results)} bai bao tu Nghien cuu Lich su.")

        except Exception as e:
            print(f"Loi: {e}")

if __name__ == "__main__":
    bot = HistoryResearch_JSON_Scraper()
    bot.run()

--- Dang quet danh muc so cu: https://vjol.info.vn/index.php/hists/issue/archive ---
Tim thay 25 so bao.
--- Dang boc tach so: 10058 ---
    + Nhà Mạc với sự nghiệp bảo vệ quyền lợi quốc gia thời kỳ đóng...
    + Nạn cướp biển và nhận thức, chủ trương phòng chống giặc biển...
    + Chương trình phổ thông cấp I cải cách 5 năm ở các tỉnh miền ...
    + Một số quan điểm cơ bản của Đại tướng Võ Nguyên Giáp về xây ...
    + Khai thác, sử dụng và quản lý gỗ trong đóng thuyền ở Trung Q...
    + Xung đột Hamas - Isreal: Tác động toàn diện và sự hình thành...
    + Tìm lại một địa danh đã mất: "Thiên Kiện sơn" trong các nguồ...
--- Dang boc tach so: 2376 ---
    + QUAN HỆ VIỆT - MỸ NỬA ĐẦU THẾ KỶ XIX...
    + THÁI ĐỘ CỦA NHẬT BÁO SÀI GÒN ĐƯƠNG THỜI VỀ SỰ KIỆN HÒA THƯỢN...
    + VỀ CẢI CÁCH HỆ THỐNG GIÁO DỤC PHỔ THÔNG NĂM 1956 Ở MIỀN BẮC ...
    + LÊ TRIỀU THÔNG SỬ CỦA LÊ QUÝ ĐÔN: MỘT SỐ VẤN ĐỀ VỀ VĂN BẢN...
    + VÀI NÉT VỀ LỊCH SỬ VIỆN TRỢ CỦA HOA KỲ Ở VIỆT NAM...
--- Dang boc tach so: 2375 --

Tạp chí Nghiên cứu Tôn Giáo (nay là Tạp chí Dân tộc học và Tôn giáo học)

In [12]:
import requests
from bs4 import BeautifulSoup
import urllib3
import time
import json

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

class ReligiousStudies_JSON_Scraper:
    def __init__(self):
        self.headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36'
        }
        self.base_url = "https://vjol.info.vn"
        self.source_name = "Tap chi Nghien cuu Ton giao"
        self.journal_code = "rsr"
        self.results = []

    def get_final_pdf(self, landing_url):
        try:
            if '/view/' in landing_url:
                return landing_url.replace('/view/', '/download/')

            res = requests.get(landing_url, headers=self.headers, verify=False, timeout=15)
            soup = BeautifulSoup(res.text, 'html.parser')

            download_btn = soup.find('a', class_='download')
            if download_btn:
                return download_btn['href']

            iframe = soup.find('iframe', src=True)
            if iframe and '.pdf' in iframe['src'].lower():
                return iframe['src']
        except:
            pass
        return None

    def run(self):
        archive_url = f"{self.base_url}/index.php/{self.journal_code}/issue/archive"
        print(f"--- Khoi dong quet {self.source_name}: {archive_url} ---")

        try:
            res_archive = requests.get(archive_url, headers=self.headers, verify=False)
            soup_archive = BeautifulSoup(res_archive.text, 'html.parser')

            issue_links = [a['href'] for a in soup_archive.find_all('a', class_='title') if '/issue/view/' in a['href']]
            print(f"Tim thay {len(issue_links)} so bao cu.")

            for issue_url in issue_links:
                print(f"--- Dang quet so: {issue_url.split('/')[-1]} ---")
                try:
                    res_issue = requests.get(issue_url, headers=self.headers, verify=False)
                    soup_issue = BeautifulSoup(res_issue.text, 'html.parser')

                    articles = soup_issue.select('.obj_article_summary')

                    for art in articles:
                        title_tag = art.select_one('.title a')
                        pdf_tag = art.select_one('a.obj_galley_link.pdf')

                        if title_tag and pdf_tag:
                            article_title = title_tag.get_text().strip()
                            landing_url = pdf_tag['href']
                            direct_pdf = self.get_final_pdf(landing_url)

                            if direct_pdf:
                                self.results.append({
                                    "source": self.source_name,
                                    "journal_code": self.journal_code,
                                    "article_title": article_title,
                                    "pdf_url": direct_pdf
                                })
                                print(f"    + {article_title[:60]}...")

                    time.sleep(0.4)
                except:
                    continue

            output_file = f"{self.journal_code}_data.json"
            with open(output_file, "w", encoding="utf-8") as f:
                json.dump(self.results, f, ensure_ascii=False, indent=4)

            print(f"XONG! Da luu {len(self.results)} bai bao vao {output_file}")

        except Exception as e:
            print(f"Loi: {e}")

if __name__ == "__main__":
    bot = ReligiousStudies_JSON_Scraper()
    bot.run()

--- Khoi dong quet Tap chi Nghien cuu Ton giao: https://vjol.info.vn/index.php/rsr/issue/archive ---
Tim thay 50 so bao cu.
--- Dang quet so: 9812 ---
    + HỆ GIÁ TRỊ QUỐC GIA VIỆT NAM - LIÊN HỆ TỚI LĨNH VỰC TÔN GIÁO...
    + CHÍNH QUYỀN CHÚA NGUYỄN VỚI SỰ HÌNH THÀNH, PHÁT TRIỂN CÁC DÒ...
    + TÌM HIỂU VỀ CƠ CHẾ VÀ HOẠT ĐỘNG CỦA HỘI ĐỒNG GIÁO PHẨM THUỘC...
    + MỘT SỐ CHUYỂN BIẾN TRONG ĐỜI SỐNG TÍN NGƯỠNG CỦA CÁC DÂN TỘC...
    + BIẾN ĐỔI NGHI LỄ RƯỚC NƯỚC TRONG TÍN NGƯỠNG THÀNH HOÀNG Ở ĐỒ...
    + MỘT SỐ HOẠT ĐỘNG TÔN GIÁO CỦA NHẤT QUÁN ĐẠO Ở VIỆT NAM HIỆN ...
    + PHẬT GIÁO, ISLAM GIÁO VÀ ĐA DẠNG TÔN GIÁO Ở NAM Á VÀ ĐÔNG NA...
--- Dang quet so: 9811 ---
    + CÔNG NHẬN CÁC TỔ CHỨC CAO ĐÀI - DẤU ẤN PHÁP NHÂN TÔN GIÁO TH...
    + TỔ CHỨC CARITAS Ở VIỆT NAM HIỆN NAY...
    + HIỆN TƯỢNG THỜ HỒ CHÍ MINH Ở VIỆT NAM HIỆN NAY VÀ VẤN ĐỀ ĐẶT...
    + THỜ HỒ CHÍ MINH TRONG MỐI LIÊN HỆ VỚI TRUYỀN THỐNG THỜ NGƯỜI...
    + HIỆN TƯỢNG THỜ CÚNG HỒ CHÍ MINH: NHẬN DIỆN VÀ MỘT VÀI NHẬN Đ...
    + C

Tạp chí Tâm lý học

In [13]:
import requests
from bs4 import BeautifulSoup
import urllib3
import time
import json

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

class PsychologyJournal_JSON_Scraper:
    def __init__(self):
        self.headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36'
        }
        self.base_url = "https://vjol.info.vn"
        self.source_name = "Tap chi Tam ly hoc"
        self.journal_code = "TLH"
        self.results = []

    def get_direct_pdf(self, landing_url):
        try:
            if '/view/' in landing_url:
                return landing_url.replace('/view/', '/download/')

            res = requests.get(landing_url, headers=self.headers, verify=False, timeout=15)
            soup = BeautifulSoup(res.text, 'html.parser')

            download_link = soup.find('a', class_='download')
            if download_link:
                return download_link['href']

            iframe = soup.find('iframe', src=True)
            if iframe and '.pdf' in iframe['src'].lower():
                return iframe['src']
        except:
            pass
        return None

    def run(self):
        archive_url = f"{self.base_url}/index.php/{self.journal_code}/issue/archive"
        print(f"--- Dang quet kho du lieu {self.source_name}: {archive_url} ---")

        try:
            res_archive = requests.get(archive_url, headers=self.headers, verify=False)
            soup_archive = BeautifulSoup(res_archive.text, 'html.parser')

            issue_links = [a['href'] for a in soup_archive.find_all('a', class_='title') if '/issue/view/' in a['href']]
            print(f"Tim thay {len(issue_links)} so bao.")

            for issue_url in issue_links:
                print(f"--- Dang quet muc luc so: {issue_url.split('/')[-1]} ---")
                try:
                    res_issue = requests.get(issue_url, headers=self.headers, verify=False)
                    soup_issue = BeautifulSoup(res_issue.text, 'html.parser')

                    articles = soup_issue.select('.obj_article_summary')

                    for art in articles:
                        title_tag = art.select_one('.title a')
                        pdf_tag = art.select_one('a.obj_galley_link.pdf')

                        if title_tag and pdf_tag:
                            article_title = title_tag.get_text().strip()
                            landing_url = pdf_tag['href']
                            direct_pdf = self.get_direct_pdf(landing_url)

                            if direct_pdf:
                                self.results.append({
                                    "source": self.source_name,
                                    "journal_code": self.journal_code,
                                    "article_title": article_title,
                                    "pdf_url": direct_pdf
                                })
                                print(f"    + {article_title[:60]}...")

                    time.sleep(0.4)
                except:
                    continue

            output_file = f"{self.journal_code.lower()}_data.json"
            with open(output_file, "w", encoding="utf-8") as f:
                json.dump(self.results, f, ensure_ascii=False, indent=4)

            print(f"HOAN THANH! Da luu {len(self.results)} bai bao vao {output_file}")

        except Exception as e:
            print(f"Loi he thong: {e}")

if __name__ == "__main__":
    bot = PsychologyJournal_JSON_Scraper()
    bot.run()

--- Dang quet kho du lieu Tap chi Tam ly hoc: https://vjol.info.vn/index.php/TLH/issue/archive ---
Tim thay 25 so bao.
--- Dang quet muc luc so: 2441 ---
    + MÔ HÌNH SÁNG TẠO CỦA TỔ CHỨC TRONG DOANH NGHIÊP VIỆT NAM...
    + VĂN HÓA KIẾM TIỀN VÀ TIÊU TIỀN CỦA THANH NIÊN HIỆN NAY...
    + NHU CẦU ÁP DỤNG THÀNH TỰU KHOA HỌC CÔNG NGHỆ TIÊN TIẾN VÀO S...
    + CÁC MỐI LIÊN HỆ XÃ HỘI CỦA NGƯỜI CAO TUỔI TẠI MỘT SỐ TRUNG T...
    + MỨC ĐỘ HỨNG THÚ HỌC TẬP CỦA SINH VIÊN TRƯỜNG ĐẠI HỌC THƯƠNG ...
    + THỰC TRẠNG HỨNG THÚ HỌC TIẾNG ANH CỦA HỌC SINH TRUNG HỌC CƠ ...
    + NHỮNG DẤU HIỆU CƠ BẢN NHẬN BIẾT HỘI CHỨNG TỰ KỶ Ở TRẺ EM...
    + ĐẶC ĐIỂM NHẬN THỨC CỦA HỌC SINH TỪ GÓC NHÌN CỦA TƯ VẤN HƯỚNG...
--- Dang quet muc luc so: 2440 ---
    + QUAN NIỆM CỦA THANH NIÊN VỀ ĐỒNG TIỀN...
    + CÁC MÔ THỨC CỦA TÍNH CÁ NHÂN – TÍNH CỘNG ĐỒNG Ở VIỆT NAM...
    + KỸ NĂNG TỔ CHỨC GIỜ GIẢNG DẠY TĂNG CƯỜNG TÍNH TÍCH CỰC CHO N...
    + MỘT SỐ BIỂU HIỆN LỐI SỐNG CỦA THANH NIÊN NÔNG THÔN HIỆN NAY...
    + NHỮNG K

Tạp chí Thông tin Khoa học xã hội

In [14]:
import requests
from bs4 import BeautifulSoup
import urllib3
import time
import json

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

class SSIR_JSON_Scraper:
    def __init__(self):
        self.headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36'
        }
        self.base_url = "https://vjol.info.vn"
        self.source_name = "Tap chi Thong tin Khoa hoc xa hoi"
        self.journal_code = "ssir"
        self.results = []

    def get_direct_pdf_link(self, landing_url):
        try:
            if '/view/' in landing_url:
                return landing_url.replace('/view/', '/download/')

            res = requests.get(landing_url, headers=self.headers, verify=False, timeout=15)
            soup = BeautifulSoup(res.text, 'html.parser')

            download_btn = soup.find('a', class_='download')
            if download_btn:
                return download_btn['href']

            iframe = soup.find('iframe', src=True)
            if iframe and '.pdf' in iframe['src'].lower():
                return iframe['src']
        except:
            pass
        return None

    def run(self):
        archive_url = f"{self.base_url}/index.php/{self.journal_code}/issue/archive"
        print(f"--- Khoi dong thu thap du lieu: {archive_url} ---")

        try:
            res_archive = requests.get(archive_url, headers=self.headers, verify=False)
            soup_archive = BeautifulSoup(res_archive.text, 'html.parser')

            issue_links = [a['href'] for a in soup_archive.find_all('a', class_='title') if '/issue/view/' in a['href']]
            print(f"Tim thay {len(issue_links)} so bao trong kho luu tru.")

            for issue_url in issue_links:
                print(f"--- Dang boc tach so: {issue_url.split('/')[-1]} ---")
                try:
                    res_issue = requests.get(issue_url, headers=self.headers, verify=False)
                    soup_issue = BeautifulSoup(res_issue.text, 'html.parser')

                    articles = soup_issue.select('.obj_article_summary')

                    for art in articles:
                        title_tag = art.select_one('.title a')
                        pdf_tag = art.select_one('a.obj_galley_link.pdf')

                        if title_tag and pdf_tag:
                            article_title = title_tag.get_text().strip()
                            landing_url = pdf_tag['href']
                            direct_link = self.get_direct_pdf_link(landing_url)

                            if direct_link:
                                self.results.append({
                                    "source": self.source_name,
                                    "journal_code": self.journal_code,
                                    "article_title": article_title,
                                    "pdf_url": direct_link
                                })
                                print(f"    + {article_title[:60]}...")

                    time.sleep(0.4)
                except Exception as e:
                    print(f"    ! Loi khi quet so bao: {e}")
                    continue

            output_file = f"{self.journal_code}_data.json"
            with open(output_file, "w", encoding="utf-8") as f:
                json.dump(self.results, f, ensure_ascii=False, indent=4)

            print(f"HOAN THANH! Da luu {len(self.results)} bai bao vao {output_file}.")

        except Exception as e:
            print(f"Loi he thong: {e}")

if __name__ == "__main__":
    bot = SSIR_JSON_Scraper()
    bot.run()

--- Khoi dong thu thap du lieu: https://vjol.info.vn/index.php/ssir/issue/archive ---
Tim thay 25 so bao trong kho luu tru.
--- Dang boc tach so: 10068 ---
    + Quan điểm của Đảng Cộng sản Việt Nam về xây dựng văn hóa bảo...
    + Trung tâm Phật giáo xứ Đông qua nghiên cứu trường hợp chùa Q...
    + Vai trò của cộng đồng trong bảo tồn lễ hội: Trường hợp Tết K...
    + Bạo lực trẻ em gái trên không gian mạng: Mặt tối của thế giớ...
    + Tác động của các yếu tố môi trường đối với sức khỏe toàn diệ...
    + Chúa Trịnh Sâm: Tiểu sử và sự nghiệp thơ ca...
    + Ứng dụng phê bình sinh thái trong nghiên cứu tiểu thuyết Việ...
    + Di tích thời Trần ở Nam Định qua tư liệu khảo cổ học từ năm ...
    + Thông tin hoạt động khoa học...
    + Giới thiệu sách mới tại Thư viện Khoa học xã hội...
    + Tổng mục lục Tạp chí Thông tin KHXH năm 2025...
--- Dang boc tach so: 10067 ---
    + Viện Hàn lâm Khoa học xã hội Việt Nam với sứ mệnh phụng sự p...
    + Xây dựng chuỗi giá trị du lịch cộng đồng củ

Tạp chí  Khoa học biến đổi khí hậu

In [15]:
import requests
from bs4 import BeautifulSoup
import urllib3
import time
import re
import json

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

class ClimateChange_JSON_Scraper:
    def __init__(self):
        self.headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36'
        }
        self.base_url = "https://vjol.info.vn"
        self.source_name = "Tap chi Khoa hoc Bien doi khi hau"
        self.journal_code = "TCKHBDKH"
        self.results = []

    def is_vietnamese(self, text):
        vietnamese_chars = re.compile(r'[àáảãạăằắẳẵặâầấẩẫậèéẻẽẹêềếểễệìíỉĩịòóỏõọôồốổỗộơờớởỡợùúủũụưừứửữựỳýỷỹỵđ]')
        return bool(vietnamese_chars.search(text.lower()))

    def get_direct_pdf(self, landing_url):
        try:
            if '/view/' in landing_url:
                return landing_url.replace('/view/', '/download/')

            res = requests.get(landing_url, headers=self.headers, verify=False, timeout=15)
            soup = BeautifulSoup(res.text, 'html.parser')
            download_link = soup.find('a', class_='download')
            if download_link:
                return download_link['href']
        except:
            pass
        return None

    def run(self):
        archive_url = f"{self.base_url}/index.php/{self.journal_code}/issue/archive"
        print(f"--- Dang quet kho du lieu {self.source_name}: {archive_url} ---")

        try:
            res_archive = requests.get(archive_url, headers=self.headers, verify=False)
            soup_archive = BeautifulSoup(res_archive.text, 'html.parser')

            issue_links = [a['href'] for a in soup_archive.find_all('a', class_='title') if '/issue/view/' in a['href']]
            print(f"Tim thay {len(issue_links)} so bao.")

            for issue_url in issue_links:
                print(f"--- Dang quet so: {issue_url.split('/')[-1]} ---")
                res_issue = requests.get(issue_url, headers=self.headers, verify=False)
                soup_issue = BeautifulSoup(res_issue.text, 'html.parser')

                articles = soup_issue.find_all('div', class_='obj_article_summary')

                for art in articles:
                    title_tag = art.find('div', class_='title')
                    if not title_tag: continue

                    article_title = title_tag.get_text().strip()

                    if self.is_vietnamese(article_title):
                        pdf_link_tag = art.find('a', class_='obj_galley_link pdf')
                        if pdf_link_tag:
                            landing_url = pdf_link_tag['href']
                            direct_pdf = self.get_direct_pdf(landing_url)

                            if direct_pdf:
                                if not any(item['pdf_url'] == direct_pdf for item in self.results):
                                    self.results.append({
                                        "source": self.source_name,
                                        "journal_code": self.journal_code,
                                        "article_title": article_title,
                                        "pdf_url": direct_pdf
                                    })
                                    print(f"    + Tieng Viet: {article_title[:60]}...")

                time.sleep(0.4)

            output_file = f"{self.journal_code.lower()}_data.json"
            with open(output_file, "w", encoding="utf-8") as f:
                json.dump(self.results, f, ensure_ascii=False, indent=4)

            print(f"HOAN THANH! Da luu {len(self.results)} bai tieng Viet vao {output_file}")

        except Exception as e:
            print(f"Loi he thong: {e}")

if __name__ == "__main__":
    bot = ClimateChange_JSON_Scraper()
    bot.run()

--- Dang quet kho du lieu Tap chi Khoa hoc Bien doi khi hau: https://vjol.info.vn/index.php/TCKHBDKH/issue/archive ---
Tim thay 25 so bao.
--- Dang quet so: 9558 ---
    + Tieng Viet: Đánh giá xu thế biến đổi khí hậu ở Thành phố Hồ Chí Minh bằn...
    + Tieng Viet: Nghiên cứu đánh giá xu thế biến đổi mưa cực đoan tại khu vực...
    + Tieng Viet: Biến đổi khí hậu và xâm nhập mặn: Giải pháp đảm bảo cấp nước...
    + Tieng Viet: Thiên tai và biến đổi khí hậu: Nhận thức và sự tiếp nhận thô...
    + Tieng Viet: Phân tích xu thế xâm nhập mặn trên hệ thống sông, kênh Tỉnh ...
    + Tieng Viet: Đánh giá diễn biến chất lượng nước mặt và xác định nguồn ô n...
    + Tieng Viet: Mối liên hệ giữa mức độ khai thác nước ngầm và hiện tượng sụ...
    + Tieng Viet: Dấu hiệu và nguy cơ ô nhiễm nhựa trong đất và trầm tích xung...
    + Tieng Viet: Nghiên cứu ứng dụng mô hình mã nguồn mở Telemac 2 chiều đánh...
    + Tieng Viet: Đặc điểm thành phần khoáng vật của cấu trúc đá basalt khu vự...
    + Tieng Vi

Tạp chí Khoa học thể thao

In [16]:
import requests
from bs4 import BeautifulSoup
import urllib3
import time
import json
from urllib.parse import urljoin

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

class SportScience_JSON_Scraper:
    def __init__(self):
        self.headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36'
        }
        self.base_url = "https://vjol.info.vn"
        self.source_name = "Tap chi Khoa hoc the thao"
        self.journal_code = "tckhthethao"
        self.results = []

    def get_file_pdf(self, landing_url):
        try:
            if '/view/' in landing_url:
                return landing_url.replace('/view/', '/download/')

            res = requests.get(landing_url, headers=self.headers, verify=False, timeout=15)
            soup = BeautifulSoup(res.text, 'html.parser')

            download_btn = soup.find('a', class_='download')
            if download_btn:
                return download_btn['href']

            iframe = soup.find('iframe', src=True)
            if iframe and '.pdf' in iframe['src'].lower():
                return iframe['src']
        except:
            pass
        return None

    def run(self):
        archive_url = f"{self.base_url}/index.php/{self.journal_code}/issue/archive"
        print(f"--- Dang quet kho du lieu {self.source_name}: {archive_url} ---")

        try:
            res_archive = requests.get(archive_url, headers=self.headers, verify=False)
            soup_archive = BeautifulSoup(res_archive.text, 'html.parser')

            issue_links = [a['href'] for a in soup_archive.find_all('a', class_='title') if '/issue/view/' in a['href']]
            print(f"Tim thay {len(issue_links)} so bao.")

            for issue_url in issue_links:
                print(f"--- Dang boc tach so: {issue_url.split('/')[-1]} ---")
                try:
                    res_issue = requests.get(issue_url, headers=self.headers, verify=False)
                    soup_issue = BeautifulSoup(res_issue.text, 'html.parser')

                    articles = soup_issue.select('.obj_article_summary')

                    for art in articles:
                        title_tag = art.select_one('.title a')
                        pdf_tag = art.select_one('a.obj_galley_link.pdf')

                        if title_tag and pdf_tag:
                            article_title = title_tag.get_text().strip()
                            landing_url = pdf_tag['href']
                            direct_pdf = self.get_file_pdf(landing_url)

                            if direct_pdf:
                                self.results.append({
                                    "source": self.source_name,
                                    "journal_code": self.journal_code,
                                    "article_title": article_title,
                                    "pdf_url": direct_pdf
                                })
                                print(f"    + {article_title[:60]}...")

                    time.sleep(0.3)
                except:
                    continue

            output_file = f"{self.journal_code}_data.json"
            with open(output_file, "w", encoding="utf-8") as f:
                json.dump(self.results, f, ensure_ascii=False, indent=4)

            print(f"HOAN THANH! Da luu {len(self.results)} bai bao vao {output_file}")

        except Exception as e:
            print(f"Loi: {e}")

if __name__ == "__main__":
    bot = SportScience_JSON_Scraper()
    bot.run()

--- Dang quet kho du lieu Tap chi Khoa hoc the thao: https://vjol.info.vn/index.php/tckhthethao/issue/archive ---
Tim thay 25 so bao.
--- Dang boc tach so: 9251 ---
    + Lời nói đầu...
    + XÂY DỰNG TIÊU CHUẨN TUYỂN CHỌN NAM VẬN ĐỘNG VIÊN VẬT TUỔI 12...
    + ĐẶC ĐIỂM QUÁ TRÌNH HỒI PHỤC CHỨC NĂNG HÔ HẤP CỦA VẬN ĐỘNG VI...
    + MỨC ĐỘ BIỂU HIỆN CỦA NHÓM GEN LIÊN QUAN ĐẾN KHẢ NĂNG KHÁNG V...
    + LỰA CHỌN CÁC TEST ĐÁNH GIÁ SỨC MẠNH TỐC ĐỘ CHO CHO NAM VẬN Đ...
    + THỰC TRẠNG CHÚ Ý CÓ CHỌN LỌC CỦA VẬN ĐỘNG VIÊN CẤP CAO, ĐỘI ...
    + THỰC TRẠNG SỬ DỤNG NƯỚC ÉP TRÁI NHÀU Ở NỮ VẬN ĐỘNG VIÊN ĐIỀN...
    + THỰC TRẠNG CÁC YẾU TỐ ẢNH HƯỞNG ĐẾN HOẠT ĐỘNG HỌC TẬP MÔN GI...
    + LỰA CHỌN MỘT SỐ TRÒ CHƠI VẬN ĐỘNG NHẰM PHÁT TRIỂN THỂ LỰC CH...
    + THỰC TRẠNG PHONG TRÀO TẬP LUYỆN BÓNG CHUYỀN TRÊN ĐỊA BÀN HUY...
    + THỰC TRẠNG GIẢNG DẠY KỸ THUẬT BƠI TRƯỜN SẤP CỦA SINH VIÊN GI...
    + ĐÁNH GIÁ THỰC TRẠNG CÔNG TÁC GIÁO DỤC THỂ CHẤT ĐỐI VỚI HỌC V...
    + LỰA CHỌN BÀI TẬP PHÁT TRIỂN THỂ LỰC CH

Tạp chí Khoa học và Công nghệ Thủy lợi

In [17]:
import requests
from bs4 import BeautifulSoup
import urllib3
import time
import json
from urllib.parse import urljoin

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

class WaterResources_JSON_Scraper:
    def __init__(self):
        self.headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36'
        }
        self.base_url = "https://vjol.info.vn"
        self.source_name = "Tap chi Khoa hoc va Cong nghe Thuy loi"
        self.journal_code = "vawr"
        self.results = []

    def get_direct_pdf(self, landing_url):
        try:
            if '/view/' in landing_url:
                return landing_url.replace('/view/', '/download/')

            res = requests.get(landing_url, headers=self.headers, verify=False, timeout=15)
            soup = BeautifulSoup(res.text, 'html.parser')

            download_btn = soup.find('a', class_='download')
            if download_btn:
                return download_btn['href']

            iframe = soup.find('iframe', src=True)
            if iframe and '.pdf' in iframe['src'].lower():
                return iframe['src']
        except:
            pass
        return None

    def run(self):
        archive_url = f"{self.base_url}/index.php/{self.journal_code}/issue/archive"
        print(f"--- Dang quet kho du lieu {self.source_name}: {archive_url} ---")

        try:
            res_archive = requests.get(archive_url, headers=self.headers, verify=False)
            soup_archive = BeautifulSoup(res_archive.text, 'html.parser')

            issue_links = [a['href'] for a in soup_archive.find_all('a', class_='title') if '/issue/view/' in a['href']]
            print(f"Tim thay {len(issue_links)} so bao.")

            for issue_url in issue_links:
                print(f"--- Dang boc tach so: {issue_url.split('/')[-1]} ---")
                try:
                    res_issue = requests.get(issue_url, headers=self.headers, verify=False)
                    soup_issue = BeautifulSoup(res_issue.text, 'html.parser')

                    articles = soup_issue.select('.obj_article_summary')

                    for art in articles:
                        title_tag = art.select_one('.title a')
                        pdf_tag = art.select_one('a.obj_galley_link.pdf')

                        if title_tag and pdf_tag:
                            article_title = title_tag.get_text().strip()
                            landing_url = pdf_tag['href']
                            direct_pdf = self.get_direct_pdf(landing_url)

                            if direct_pdf:
                                self.results.append({
                                    "source": self.source_name,
                                    "journal_code": self.journal_code,
                                    "article_title": article_title,
                                    "pdf_url": direct_pdf
                                })
                                print(f"    + {article_title[:60]}...")

                    time.sleep(0.3)
                except:
                    continue

            output_file = f"{self.journal_code}_data.json"
            with open(output_file, "w", encoding="utf-8") as f:
                json.dump(self.results, f, ensure_ascii=False, indent=4)

            print(f"HOAN THANH! Da luu {len(self.results)} bai bao vao {output_file}")

        except Exception as e:
            print(f"Loi: {e}")

if __name__ == "__main__":
    bot = WaterResources_JSON_Scraper()
    bot.run()

--- Dang quet kho du lieu Tap chi Khoa hoc va Cong nghe Thuy loi: https://vjol.info.vn/index.php/vawr/issue/archive ---
Tim thay 25 so bao.
--- Dang boc tach so: 9118 ---
    + Mô phỏng xâm nhập mặn ở đồng bằng sông Cửu Long trong điề...
    + Kết quả thí nghiệm xác định nguyên nhân nứt thân đê sông khu...
    + Tích hợp mô hình khí hậu khu vực REGCM và mô hình thuỷ văn W...
    + Phân tích, xác định nguyên nhân gây sạt lở bờ cồn Phú Đa, hu...
    + Nghiên cứu ảnh hưởng của hàm lượng tro bay (FA) và magiê oxy...
    + Hiện trạng nuôi trồng thủy sản kết hợp phát triển rừng ngập ...
    + Thay đổi mưa và nhiệt độ ở tỉnh hậu giang trong điều kiện bi...
    + Nghiên cứu khảo nghiệm ảnh hưởng của tưới nước mặn đến sinh ...
    + Giải pháp công trình bảo vệ bờ biển tỉnh Bình Thuận...
    + Đề xuất các giải pháp giảm thiểu ngập úng trên địa bàn tỉnh ...
    + Đặc tính dễ tổn thương sinh kế và thu nhập của hộ nuôi trồng...
    + Phân vùng chất lượng nước sông hương và phá tam giang dưới t..

Tạp chí Nghiên cứu và Phát triển

In [18]:
import requests
from bs4 import BeautifulSoup
import urllib3
import time
import json
from urllib.parse import urljoin

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

class NCPT_Hue_JSON_Scraper:
    def __init__(self):
        self.headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36'
        }
        self.base_url = "https://vjol.info.vn"
        self.source_name = "Tap chi Nghien cuu va Phat trien"
        self.journal_code = "ncpt-hue"
        self.results = []

    def get_direct_pdf(self, landing_url):
        try:
            if '/view/' in landing_url:
                return landing_url.replace('/view/', '/download/')

            res = requests.get(landing_url, headers=self.headers, verify=False, timeout=15)
            soup = BeautifulSoup(res.text, 'html.parser')

            download_btn = soup.find('a', class_='download')
            if download_btn:
                return download_btn['href']

            iframe = soup.find('iframe', src=True)
            if iframe and '.pdf' in iframe['src'].lower():
                return iframe['src']
        except:
            pass
        return None

    def run(self):
        archive_url = f"{self.base_url}/index.php/{self.journal_code}/issue/archive"
        print(f"--- Dang quet kho du lieu {self.source_name}: {archive_url} ---")

        try:
            res_archive = requests.get(archive_url, headers=self.headers, verify=False)
            soup_archive = BeautifulSoup(res_archive.text, 'html.parser')

            issue_links = [a['href'] for a in soup_archive.find_all('a', class_='title') if '/issue/view/' in a['href']]
            print(f"Tim thay {len(issue_links)} so bao.")

            for issue_url in issue_links:
                print(f"--- Dang quet so: {issue_url.split('/')[-1]} ---")
                try:
                    res_issue = requests.get(issue_url, headers=self.headers, verify=False)
                    soup_issue = BeautifulSoup(res_issue.text, 'html.parser')

                    articles = soup_issue.select('.obj_article_summary')

                    for art in articles:
                        title_tag = art.select_one('.title a')
                        pdf_tag = art.select_one('a.obj_galley_link.pdf')

                        if title_tag and pdf_tag:
                            article_title = title_tag.get_text().strip()
                            landing_url = pdf_tag['href']
                            direct_pdf = self.get_direct_pdf(landing_url)

                            if direct_pdf:
                                self.results.append({
                                    "source": self.source_name,
                                    "journal_code": self.journal_code,
                                    "article_title": article_title,
                                    "pdf_url": direct_pdf
                                })
                                print(f"    + {article_title[:60]}...")

                    time.sleep(0.3)
                except Exception as e:
                    print(f"    ! Loi tai so bao nay: {e}")
                    continue

            output_file = f"{self.journal_code.replace('-', '_')}_data.json"
            with open(output_file, "w", encoding="utf-8") as f:
                json.dump(self.results, f, ensure_ascii=False, indent=4)

            print(f"XONG! Thu hoach duoc {len(self.results)} bai bao tu {self.source_name}.")

        except Exception as e:
            print(f"Loi he thong: {e}")

if __name__ == "__main__":
    bot = NCPT_Hue_JSON_Scraper()
    bot.run()

--- Dang quet kho du lieu Tap chi Nghien cuu va Phat trien: https://vjol.info.vn/index.php/ncpt-hue/issue/archive ---
Tim thay 25 so bao.
--- Dang quet so: 8090 ---
--- Dang quet so: 8023 ---
--- Dang quet so: 5400 ---
    + Lê Quang Định và sách Hoàng Việt nhất thống dư địa chí (Phần...
    + Điện Thái Hòa và Điện Cần Chánh thời Nguyễn nhìn từ góc độ b...
    + Phong trào Đồng khởi ở Thừa Thiên Huế từ năm 1964 đến đầu nă...
    + Xây dựng và quản lý chỉ dẫn địa lý cho sản phẩm Dầu tràm Huế...
    + Khoa học và Công nghệ là động lực thúc đẩy phát triển kinh t...
    + Biện thể tiểu thuyết Hán văn Việt Nam: Vấn đề tiểu thuyết tr...
    + Vụ án đánh roi....
    + Người sáng chế ra chiếc Xích lô ở châu Á là người Pháp....
    + Nhịp sống Huế...
--- Dang quet so: 5399 ---
    + Lời Tòa soạn....
    + Đồ pháp lang cốt kim loại ở Cố Cung Bắc Kinh....
    + Tổng quan về pháp lam và nhận thức mới về Pháp lam Huế thời ...
    + Pháp lam Huế - di sản độc đáo của triều Nguyễn....
    + Trang trí 

Tạp chí Khoa học và Công nghệ - Đại học Thái Nguyên

In [19]:
import requests
from bs4 import BeautifulSoup
import urllib3
import time
import json
from urllib.parse import urljoin

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

class TNU_JSON_Scraper:
    def __init__(self):
        self.headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36'
        }
        self.base_url = "https://vjol.info.vn"
        self.source_name = "Tap chi Khoa hoc & Cong nghe - DH Thai Nguyen"
        self.journal_code = "tnu"
        self.results = []

    def get_direct_pdf(self, landing_url):
        try:
            if '/view/' in landing_url:
                return landing_url.replace('/view/', '/download/')

            res = requests.get(landing_url, headers=self.headers, verify=False, timeout=15)
            soup = BeautifulSoup(res.text, 'html.parser')

            download_btn = soup.find('a', class_='download')
            if download_btn:
                return download_btn['href']
        except:
            pass
        return None

    def run(self):
        archive_url = f"{self.base_url}/index.php/{self.journal_code}/issue/archive"
        print(f"--- Bat dau quet du lieu {self.source_name}: {archive_url} ---")

        try:
            res_archive = requests.get(archive_url, headers=self.headers, verify=False)
            soup_archive = BeautifulSoup(res_archive.text, 'html.parser')

            issue_links = [a['href'] for a in soup_archive.find_all('a', class_='title') if '/issue/view/' in a['href']]
            print(f"Tim thay {len(issue_links)} so bao trong kho luu tru.")

            for issue_url in issue_links:
                print(f"--- Dang quet so: {issue_url.split('/')[-1]} ---")
                try:
                    res_issue = requests.get(issue_url, headers=self.headers, verify=False)
                    soup_issue = BeautifulSoup(res_issue.text, 'html.parser')

                    articles = soup_issue.select('.obj_article_summary')

                    for art in articles:
                        title_tag = art.select_one('.title a')
                        pdf_tag = art.select_one('a.obj_galley_link.pdf')

                        if title_tag and pdf_tag:
                            article_title = title_tag.get_text().strip()
                            landing_url = pdf_tag['href']
                            direct_pdf = self.get_direct_pdf(landing_url)

                            if direct_pdf:
                                self.results.append({
                                    "source": self.source_name,
                                    "journal_code": self.journal_code,
                                    "article_title": article_title,
                                    "pdf_url": direct_pdf
                                })
                                print(f"    + {article_title[:65]}...")

                    time.sleep(0.2)
                except Exception as e:
                    print(f"    ! Loi tai so bao: {e}")
                    continue

            output_file = f"{self.journal_code}_data.json"
            with open(output_file, "w", encoding="utf-8") as f:
                json.dump(self.results, f, ensure_ascii=False, indent=4)

            print(f"XONG! Da thu hoach duoc {len(self.results)} bai bao.")

        except Exception as e:
            print(f"Loi he thong: {e}")

if __name__ == "__main__":
    bot = TNU_JSON_Scraper()
    bot.run()

--- Bat dau quet du lieu Tap chi Khoa hoc & Cong nghe - DH Thai Nguyen: https://vjol.info.vn/index.php/tnu/issue/archive ---
Tim thay 25 so bao trong kho luu tru.
--- Dang quet so: 10216 ---
    + HAI THẬP KỶ HỢP TÁC GIỮA TỈNH THÁI NGUYÊN (VIỆT NAM) VÀ TỈNH GYEO...
    + NÂNG CAO CHẤT LƯỢNG GIẢNG DẠY MÔN HỌC GIÁO DỤC CHÍNH TRỊ TẠI TRƯỜ...
    + NÂNG CAO CHẤT LƯỢNG ĐÀO TẠO NGUỒN NHÂN LỰC TẠI TRƯỜNG ĐẠI HỌC KIN...
    + THUẬN LỢI VÀ KHÓ KHĂN KHI TÍCH HỢP VIỆC SỬ DỤNG CHATGPT  VÀO HỌC ...
    + KHẢO SÁT SO SÁNH NGỮ NGHĨA VÀ HÀM Ý VĂN HÓA CỦA CÁC THÀNH NGỮ CHỨ...
    + THÍCH ỨNG VÀ GIẢM THIỂU TÁC ĐỘNG CỦA BIẾN ĐỔI KHÍ HẬU  ĐỐI VỚI NÔ...
    + ĐÁNH GIÁ TIỀM NĂNG PHÁT TRIỂN DU LỊCH NET ZERO TẠI CỤM ĐIỂM DU LỊ...
    + THẾ GIỚI QUAN VÀ ĐỜI SỐNG KINH TẾ - XÃ HỘI CỦA NGƯỜI THÁI QUA SỬ ...
    + THỰC TRẠNG NHU CẦU HỌC TẬP VÀ ĐIỀU KIỆN ĐẢM BẢO GIẢNG DẠY MÔN PIC...
    + ĐẢNG LAO ĐỘNG VIỆT NAM LÃNH ĐẠO CÔNG TÁC BÁO CHÍ (1955-1964)...
    + BẢO TỒN VÀ PHÁT HUY GIÁ TRỊ VĂN HÓA DÂN TỘC SÁN CHAY XÓM Đ

Tạp chí KHCN xây dựng

In [20]:
import requests
from bs4 import BeautifulSoup
import urllib3
import time
import json
from urllib.parse import urljoin

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

class STCE_HUCE_JSON_Scraper:
    def __init__(self):
        self.headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36'
        }
        self.base_url = "https://stce.huce.edu.vn"
        self.source_name = "Tap chi Khoa hoc Cong nghe Xay dung (HUCE)"
        self.journal_code = "stce"
        self.results = []

    def get_direct_pdf(self, landing_url):
        try:
            if '/view/' in landing_url:
                return landing_url.replace('/view/', '/download/')

            res = requests.get(landing_url, headers=self.headers, verify=False, timeout=20)
            soup = BeautifulSoup(res.text, 'html.parser')

            download_btn = soup.find('a', class_='download')
            if download_btn:
                return download_btn['href']

            fallback = soup.find('a', href=lambda x: x and '/article/download/' in x)
            if fallback:
                return fallback['href']
        except:
            pass
        return None

    def run(self):
        archive_url = f"{self.base_url}/index.php/vn/issue/archive"
        print(f"--- Ket noi toi {self.source_name}: {archive_url} ---")

        try:
            res_archive = requests.get(archive_url, headers=self.headers, verify=False)
            soup_archive = BeautifulSoup(res_archive.text, 'html.parser')

            issue_links = []
            for a in soup_archive.find_all('a', class_='title'):
                href = a['href']
                if '/issue/view/' in href:
                    issue_links.append(urljoin(self.base_url, href))

            issue_links = list(set(issue_links))
            print(f"Tim thay {len(issue_links)} so bao.")

            for issue_url in issue_links:
                print(f"--- Dang quet muc luc so: {issue_url.split('/')[-1]} ---")
                try:
                    res_issue = requests.get(issue_url, headers=self.headers, verify=False)
                    soup_issue = BeautifulSoup(res_issue.text, 'html.parser')

                    articles = soup_issue.select('.obj_article_summary')

                    for art in articles:
                        title_tag = art.select_one('.title a')
                        pdf_tag = art.select_one('a.obj_galley_link.pdf')

                        if title_tag and pdf_tag:
                            article_title = title_tag.get_text().strip()
                            landing_url = urljoin(self.base_url, pdf_tag['href'])
                            direct_pdf = self.get_direct_pdf(landing_url)

                            if direct_pdf:
                                self.results.append({
                                    "source": self.source_name,
                                    "journal_code": self.journal_code,
                                    "article_title": article_title,
                                    "pdf_url": direct_pdf
                                })
                                print(f"    + {article_title[:65]}...")

                    time.sleep(0.3)
                except Exception as e:
                    print(f"    ! Loi tai so bao nay: {e}")
                    continue

            output_file = f"{self.journal_code}_huce_data.json"
            with open(output_file, "w", encoding="utf-8") as f:
                json.dump(self.results, f, ensure_ascii=False, indent=4)

            print(f"HOAN THANH! Da luu {len(self.results)} bai bao vao {output_file}")

        except Exception as e:
            print(f"Loi he thong: {e}")

if __name__ == "__main__":
    bot = STCE_HUCE_JSON_Scraper()
    bot.run()

--- Ket noi toi Tap chi Khoa hoc Cong nghe Xay dung (HUCE): https://stce.huce.edu.vn/index.php/vn/issue/archive ---
Tim thay 10 so bao.
--- Dang quet muc luc so: 122 ---
    + Table of Contents...
    + Dao động riêng và đáp ứng động của tấm FGM có vi bọt rỗng trên nề...
    + Dao động cưỡng bức của dầm nano làm bằng vật liệu FGM có vết nứt...
    + Phân tích dao động riêng của kết cấu tấm FGM có kích thước micro ...
    + Phân tích tĩnh dầm cong sandwich FGM xốp đặt trên nền đàn hồi Win...
    + Chẩn đoán vết nứt trong kết cấu tấm bê tông cốt thép sử dụng các ...
    + Tính toán thiết kế cột thép cho nhà nhiều tầng chịu tải trọng gió...
    + Nghiên cứu ứng xử chịu xoắn của dầm bê tông cốt thép trước và sau...
    + Xác định các thông số cơ bản của máy khuấy tạo bông bùn trong tổ ...
    + Sức chịu tải của cọc khoan nhồi trong đá: Một nghiên cứu điển hìn...
    + Nghiên cứu xây dựng mô hình phân tích dự đoán ứng xử chịu uốn của...
    + Phân tích phi tuyến tĩnh dầm có cơ tính biến thi

Tạp chí Khoa học Sức khỏe

In [21]:
import requests
from bs4 import BeautifulSoup
import urllib3
import time
import json
from urllib.parse import urljoin

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

class HPMU_Health_JSON_Scraper:
    def __init__(self):
        self.headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36'
        }
        self.base_url = "https://vjol.info.vn"
        self.source_name = "Tap chi Khoa hoc Suc khoe (HPMU)"
        self.journal_code = "khoahocsuckhoe-HPMU"
        self.results = []

    def get_direct_pdf(self, landing_url):
        try:
            if '/view/' in landing_url:
                return landing_url.replace('/view/', '/download/')

            res = requests.get(landing_url, headers=self.headers, verify=False, timeout=15)
            soup = BeautifulSoup(res.text, 'html.parser')

            download_btn = soup.find('a', class_='download')
            if download_btn:
                return download_btn['href']

            iframe = soup.find('iframe', src=True)
            if iframe and '.pdf' in iframe['src'].lower():
                return iframe['src']
        except:
            pass
        return None

    def run(self):
        archive_url = f"{self.base_url}/index.php/{self.journal_code}/issue/archive"
        print(f"--- Dang quet kho du lieu {self.source_name}: {archive_url} ---")

        try:
            res_archive = requests.get(archive_url, headers=self.headers, verify=False)
            soup_archive = BeautifulSoup(res_archive.text, 'html.parser')

            issue_links = [a['href'] for a in soup_archive.find_all('a', class_='title') if '/issue/view/' in a['href']]
            print(f"Tim thay {len(issue_links)} so bao.")

            for issue_url in issue_links:
                print(f"--- Quet so: {issue_url.split('/')[-1]} ---")
                try:
                    res_issue = requests.get(issue_url, headers=self.headers, verify=False)
                    soup_issue = BeautifulSoup(res_issue.text, 'html.parser')

                    articles = soup_issue.select('.obj_article_summary')

                    for art in articles:
                        title_tag = art.select_one('.title a')
                        pdf_tag = art.select_one('a.obj_galley_link.pdf')

                        if title_tag and pdf_tag:
                            article_title = title_tag.get_text().strip()
                            landing_url = pdf_tag['href']
                            direct_pdf = self.get_direct_pdf(landing_url)

                            if direct_pdf:
                                self.results.append({
                                    "source": self.source_name,
                                    "journal_code": self.journal_code,
                                    "article_title": article_title,
                                    "pdf_url": direct_pdf
                                })
                                print(f"    + {article_title[:65]}...")

                    time.sleep(0.2)
                except Exception as e:
                    print(f"    ! Loi tai so nay: {e}")
                    continue

            output_file = f"{self.journal_code.lower().replace('-', '_')}_data.json"
            with open(output_file, "w", encoding="utf-8") as f:
                json.dump(self.results, f, ensure_ascii=False, indent=4)

            print(f"XONG! Da luu {len(self.results)} bai bao vao {output_file}")

        except Exception as e:
            print(f"Loi he thong: {e}")

if __name__ == "__main__":
    bot = HPMU_Health_JSON_Scraper()
    bot.run()

--- Dang quet kho du lieu Tap chi Khoa hoc Suc khoe (HPMU): https://vjol.info.vn/index.php/khoahocsuckhoe-HPMU/issue/archive ---
Tim thay 12 so bao.
--- Quet so: 9549 ---
    + Mục lục Tập 3, số 5, 2025...
    + Khảo sát hiệu quả áp dụng công cụ điều chỉnh liều dựa trên chức n...
    + Phân tích thực trạng sử dụng thuốc điều trị tăng huyết áp tại phò...
    + Ứng dụng kỹ thuật Multiplex PCR trong chẩn đoán một số căn nguyên...
    + Thực trạng nhu cầu học sau đại học của sinh viên Răng Hàm Mặt hai...
    + Kiến thức, thái độ và thực hành của bà mẹ về việc bổ sung vitamin...
    + Thực trạng bệnh tăng huyết áp và một số yếu tố liên quan ở người ...
    + Thực trạng cận thị và một số yếu tố liên quan của sinh viên y đa ...
    + Hoạt tính của tế bào diệt tự nhiên NK ở người trưởng thành có rối...
    + Thực trạng và một số yếu tố liên quan đến giảm ham muốn tình dục ...
    + Kiến thức về phòng hộ cá nhân của sinh viên điều dưỡng chính quy ...
    + Đặc điểm nội soi đường hô hấp trên khi

In [23]:
import json
import os

file_paths = [
    '/content/hists_data.json', '/content/khoahocsuckhoe_hpmu_data.json',
    '/content/khoahoctre_final.json', '/content/khxh_mientrung_data.json',
    '/content/khxhvn_data.json', '/content/ncpt_hue_data.json',
    '/content/rsr_data.json', '/content/ssir_data.json',
    '/content/stce_huce_data.json', '/content/tcgd_data.json',
    '/content/tckhbdkh_data.json', '/content/tckhthethao_data.json',
    '/content/tlh_data.json', '/content/tnu_data.json',
    '/content/vawr_data.json', '/content/vjst_b_data.json',
    '/content/vrtc_do_ben_data.json', '/content/vrtc_sinh_thai_data.json',
    '/content/vrtc_data.json'
]

combined_data = []

print("--- Bat dau qua trinh gop file ---")

for path in file_paths:
    if os.path.exists(path):
        try:
            with open(path, 'r', encoding='utf-8') as f:
                data = json.load(f)

                if isinstance(data, list):
                    combined_data.extend(data)
                else:
                    combined_data.append(data)
            print(f"OK: {path}")
        except Exception as e:
            print(f"Loi: {path} - {e}")
    else:
        print(f"Khong tim thay: {path}")

# Lay so luong object de dat ten file
total_objects = len(combined_data)
output_file = f'/content/{total_objects}_data.json'

with open(output_file, 'w', encoding='utf-8') as f:
    json.dump(combined_data, f, ensure_ascii=False, indent=4)

print(f"\nHoan tat! Da gop {total_objects} doi tuong.")
print(f"File tong hop da duoc luu tai: {output_file}")

--- Bat dau qua trinh gop file ---
OK: /content/hists_data.json
OK: /content/khoahocsuckhoe_hpmu_data.json
OK: /content/khoahoctre_final.json
OK: /content/khxh_mientrung_data.json
OK: /content/khxhvn_data.json
OK: /content/ncpt_hue_data.json
OK: /content/rsr_data.json
OK: /content/ssir_data.json
OK: /content/stce_huce_data.json
OK: /content/tcgd_data.json
OK: /content/tckhbdkh_data.json
OK: /content/tckhthethao_data.json
OK: /content/tlh_data.json
OK: /content/tnu_data.json
OK: /content/vawr_data.json
OK: /content/vjst_b_data.json
OK: /content/vrtc_do_ben_data.json
OK: /content/vrtc_sinh_thai_data.json
OK: /content/vrtc_data.json

Hoan tat! Da gop 6089 doi tuong.
File tong hop da duoc luu tai: /content/6089_data.json


In [24]:
import json
import re
import os

# Đường dẫn file đầu vào
input_file = '/content/6089_data.json'

# Biểu thức chính quy để tìm ký tự tiếng Nga (bảng chữ cái Cyrillic)
russian_pattern = re.compile(r'[\u0400-\u04FF]')

try:
    if not os.path.exists(input_file):
        print(f"Khong tim thay file: {input_file}")
    else:
        with open(input_file, 'r', encoding='utf-8') as f:
            data = json.load(f)

        if isinstance(data, list):
            # Loc: Chi giu lai cac object KHONG co ky tu tieng Nga trong article_title
            cleaned_data = [
                obj for obj in data
                if not (isinstance(obj.get('article_title'), str) and russian_pattern.search(obj['article_title']))
            ]

            # Dem so luong
            original_count = len(data)
            final_count = len(cleaned_data)
            removed_count = original_count - final_count

            # Dat ten file theo format moi
            output_file = f'/content/{final_count}_data.json'

            # Luu file
            with open(output_file, 'w', encoding='utf-8') as f:
                json.dump(cleaned_data, f, ensure_ascii=False, indent=4)

            print(f"--- Ket qua lam sach ---")
            print(f"Tong so ban dau: {original_count}")
            print(f"Da loai bo (tieng Nga): {removed_count}")
            print(f"Con lai: {final_count}")
            print(f"File sach da luu tai: {output_file}")
        else:
            print("Canh bao: Dinh dang file khong phai la List.")

except Exception as e:
    print(f"Loi: {e}")

--- Ket qua lam sach ---
Tong so ban dau: 6089
Da loai bo (tieng Nga): 32
Con lai: 6057
File sach da luu tai: /content/6057_data.json


In [25]:
import json
import os

input_file = '/content/6057_data.json'

try:
    if not os.path.exists(input_file):
        print(f"Khong tim thay file: {input_file}")
    else:
        with open(input_file, 'r', encoding='utf-8') as f:
            data = json.load(f)

        if isinstance(data, list):
            total_records = len(data)

            # Su dung Dictionary de theo doi trung lap theo tieu de
            # Key: article_title, Value: list cac object co cung tieu de do
            seen_titles = {}
            duplicates = []

            for obj in data:
                title = obj.get('article_title', '').strip()
                if title in seen_titles:
                    duplicates.append(obj)
                    seen_titles[title].append(obj)
                else:
                    seen_titles[title] = [obj]

            # Loc ra nhung nhom thuc su bi trung
            duplicate_groups = {k: v for k, v in seen_titles.items() if len(v) > 1}

            print(f"--- Ket qua kiem tra trung lap ---")
            print(f"Tong so ban ghi kiem tra: {total_records}")
            print(f"So luong tieu de duy nhat: {len(seen_titles)}")
            print(f"So luong ban ghi bi trung (du thua): {len(duplicates)}")
            print(f"So nhom tieu de co ton tai trung lap: {len(duplicate_groups)}")

            # In ra thu 3 vi du ve cac bai bao bi trung (neu co)
            if duplicate_groups:
                print(f"\n--- Vi du cac muc bi trung lap (toi da 3): ---")
                for i, (title, group) in enumerate(list(duplicate_groups.items())[:3]):
                    print(f"{i+1}. Tieu de: {title}")
                    print(f"   => Xuat hien {len(group)} lan.")
                    for item in group:
                        print(f"      - Nguon: {item.get('source')} | PDF: {item.get('pdf_url')}")
            else:
                print("\nChuc mung! Khong tim thay doi tuong nao bi trung lap.")
        else:
            print("Canh bao: Du lieu khong phai dang List.")

except Exception as e:
    print(f"Loi: {e}")

--- Ket qua kiem tra trung lap ---
Tong so ban ghi kiem tra: 6057
So luong tieu de duy nhat: 5965
So luong ban ghi bi trung (du thua): 92
So nhom tieu de co ton tai trung lap: 19

--- Vi du cac muc bi trung lap (toi da 3): ---
1. Tieu de: CÁC THƯƠNG CẢNG VÙNG NGHỆ - TĨNH VÀ GIAO THƯƠNG KHU VỰC THẾ KỶ XI-XIV
   => Xuat hien 2 lan.
      - Nguon: Tap chi Nghien cuu Lich su | PDF: https://vjol.info.vn/index.php/hists/article/download/21412/18325
      - Nguon: Tap chi Nghien cuu Lich su | PDF: https://vjol.info.vn/index.php/hists/article/download/20970/18279
2. Tieu de: Mục lục
   => Xuat hien 3 lan.
      - Nguon: Tap chi Khoa hoc Suc khoe (HPMU) | PDF: https://vjol.info.vn/index.php/khoahocsuckhoe-HPMU/article/download/102343/86266
      - Nguon: Tap chi Khoa hoc Suc khoe (HPMU) | PDF: https://vjol.info.vn/index.php/khoahocsuckhoe-HPMU/article/download/102009/86011
      - Nguon: Tap chi Khoa hoc Suc khoe (HPMU) | PDF: https://vjol.info.vn/index.php/khoahocsuckhoe-HPMU/article/download/

In [26]:
import json
import os

input_file = '/content/6057_data.json'

# Danh sách các từ khóa tiêu đề cần loại bỏ (blacklisted titles)
blacklist = [
    "Mục lục", "Untitled Article", "Thông tin tòa soạn",
    "Lời nói đầu", "Thể lệ gửi bài", "Bìa", "Phụ lục"
]

try:
    if not os.path.exists(input_file):
        print(f"Khong tim thay file: {input_file}")
    else:
        with open(input_file, 'r', encoding='utf-8') as f:
            data = json.load(f)

        if isinstance(data, list):
            original_count = len(data)

            seen_titles = set()
            cleaned_data = []
            junk_count = 0
            dup_count = 0

            for obj in data:
                title = obj.get('article_title', '').strip()
                title_upper = title.upper() # Chuyển về chữ hoa để so sánh chính xác hơn

                # 1. Kiểm tra nếu tiêu đề nằm trong danh sách đen
                is_junk = any(bad_word.upper() in title_upper for bad_word in blacklist)

                if is_junk:
                    junk_count += 1
                    continue

                # 2. Kiểm tra trùng lặp
                if title_upper not in seen_titles:
                    cleaned_data.append(obj)
                    seen_titles.add(title_upper)
                else:
                    dup_count += 1

            final_count = len(cleaned_data)
            output_file = f'/content/{final_count}_data.json'

            # Lưu file đã làm sạch
            with open(output_file, 'w', encoding='utf-8') as f:
                json.dump(cleaned_data, f, ensure_ascii=False, indent=4)

            print(f"--- Ket qua lam sach triet de ---")
            print(f"Tong so ban dau: {original_count}")
            print(f"Da loai bo trung lap: {dup_count}")
            print(f"Da loai bo tieu de rac (Muc luc, Untitled...): {junk_count}")
            print(f"Tong so con lai: {final_count}")
            print(f"File sạch da luu tai: {output_file}")

        else:
            print("Canh bao: Du lieu khong phai dang List.")

except Exception as e:
    print(f"Loi: {e}")

--- Ket qua lam sach triet de ---
Tong so ban dau: 6057
Da loai bo trung lap: 74
Da loai bo tieu de rac (Muc luc, Untitled...): 44
Tong so con lai: 5939
File sạch da luu tai: /content/5939_data.json


In [28]:
import json
import os
from collections import Counter

input_file = '/content/5939_data.json'

try:
    if not os.path.exists(input_file):
        print(f"Khong tim thay file: {input_file}")
    else:
        with open(input_file, 'r', encoding='utf-8') as f:
            data = json.load(f)

        if isinstance(data, list):
            titles = [str(obj.get('article_title', '')).strip().upper() for obj in data]

            total_count = len(titles)
            unique_count = len(set(titles))
            duplicate_count = total_count - unique_count

            print(f"--- Kiem tra cuoi cung ---")
            print(f"File kiem tra: {input_file}")
            print(f"Tong so doi tuong: {total_count}")

            if duplicate_count == 0:
                print("Ket qua: Khong con tieu de nao bi trung lap.")
            else:
                print(f"Canh bao: Van con {duplicate_count} tieu de trung lap!")

                counts = Counter(titles)
                dups = [t for t, c in counts.items() if c > 1]
                for d in dups[:5]:
                    print(f"   - Trung: {d}")
        else:
            print("Loi: Du lieu khong phai dang List.")
except Exception as e:
    print(f"Loi: {e}")

--- Kiem tra cuoi cung ---
File kiem tra: /content/5939_data.json
Tong so doi tuong: 5939
Ket qua: Khong con tieu de nao bi trung lap.
